![alt text](DataKind_orange.png)

### Performs grid search for binary classification of Stabilization and Royalties clauses

In [39]:
import os, io
from collections import Counter
from itertools import product
import requests
import re, string
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize, pos_tag
from textblob import TextBlob
from textstat.textstat import textstat
from langdetect import detect

from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [2]:
resource_folder = 'contract_data/Contracts_Annotations/resource_contracts/'
land_folder = 'contract_data/Contracts_Annotations/openland_contracts/'
folders = [resource_folder,land_folder]

In [3]:
labels = ['stabilization','royalties']
n_folds = 5 

In [8]:
# Languages must be supported by NLTK Snowball Stemmer and stopwords
languages = ['english','french','spanish','italian','portuguese','dutch','swedish']
s = requests.get('http://data.okfn.org/data/core/language-codes/r/language-codes.csv').content
language_codes = pd.read_csv(io.StringIO(s.decode('utf-8')))
language_codes.columns = ['language_code','Language']

In [9]:
annotations = pd.DataFrame()
for folder in folders:
    xls_files = [f for f in os.listdir(folder) if f.lower().endswith('.xls')]
    for xls in xls_files:
        temp = pd.read_excel(folder + xls)
        if len(temp) > 0:
            temp['OCID'] = xls[:-4]
            temp['Source'] = folder.split('/')[-2]
            annotations = annotations.append(temp)
print "Number of annotations: " + str(len(annotations))

Number of annotations: 20609


In [10]:
# drop blank annotations
annotations.dropna(subset=['Annotation Text'],inplace=True)
print len(annotations)
# If duplicate text appears within the same contract, drop it
annotations.drop_duplicates(['Annotation Text','OCID','Category'],inplace=True)
print len(annotations)

20561
16364


In [11]:
labels = [label.lower() for label in labels]
annotations['label'] = [x.lower() if x.lower() in labels else 'other' for x in annotations['Category']]

In [12]:
# Given some duplicate text with different Category labels, keep the labels that are in our target label list when dropping
sort_num = range(len(labels) + 1)
sort_key = dict(zip(labels,sort_num))
sort_key['other'] = sort_num[-1]
annotations['sort_key'] = [sort_key[x] for x in annotations['label']]
annotations.sort_values(by='sort_key',inplace=True,ascending=True)
annotations.drop_duplicates(['Annotation Text','OCID'],keep='first',inplace=True)
print len(annotations)

16181


In [13]:
y = list(annotations['label'])
yunique = list(np.unique(y))
for item in yunique:
    print str("{0:.2f}%".format(100*y.count(item) / float(len(y)))) + " " + item

96.40% other
2.07% royalties
1.53% stabilization


In [14]:
annotations.head(2)

,Category,Topic,Annotation Text,PDF Page Number,Article Reference,OCID,Source,label,sort_key
23,Stabilization,Legal Rules,"If, as a result of any change in the laws, rul...",30,Art. 18. 3,ocds-591adf-1990915353,resource_contracts,stabilization,0
29,Stabilization,Legal Rules,If a material change occurs to Phillips China ...,40,Art. 27.2,ocds-591adf-1706855956,resource_contracts,stabilization,0


In [19]:
def punctuation_remove(text):
    """
    Mutates and returns text where all punctuation besides underscores,
    are replaced
    """
    punctuation_re = r'[^\w\s_]'
    new_text = nltk.regexp.re.sub(punctuation_re, ' ', text)
    return new_text

def replace_numbers(text):
    ''' 
    Removes all characters but periods, commas and alpha-numeric and 
    returns all numeric values replace with the word numeric_value
    '''
    allowed = {",", "."," "}.union(string.ascii_letters).union([str(num) for num in range(0,10)])
    filtered = ''.join([character for character in text if character in allowed])
    for word in text.split():
        try:
            int(word.replace(',','').replace('.',''))
            text = text.replace(word,'numericvalue')
        except:
            pass
    return text

def perform_lowercase(text):
    """
    Mutates and returns text where all characters are lowercased
    """
    try:
        new_text = text.lower()
    except:
        new_text = str(text).lower()
    return new_text

def underscore_remove(text):
    '''
    replaces multiple underscores with text fillintheblank
    and single underscore with space
    '''
    double_underscore_re = r'(__[a-zA-Z0-9_]*(__)?)'
    text = nltk.regexp.re.sub(double_underscore_re,'fillintheblank',text)
    return text.replace('_',' ')

def doublespace_remove(text):
    return re.sub(' +',' ',text)

def textblobsent(text):
    '''
    returns the TextBlob polarity and subjectivity
    '''
    text = text.encode('ascii','replace')
    sent = TextBlob(text).sentiment
    return pd.Series([sent.polarity,sent.subjectivity])

def get_avg_wordlength(document):
    wordlengths = [len(word) for word in document.split()]
    if len(wordlengths) == 0:
        return 0
    else:
        return np.mean(wordlengths)

def determine_tense(essay):
    '''
    Returns the number of past, present and future tense verbs in a given text
    '''
    text = word_tokenize(essay)
    tagged = pos_tag(text)

    numfuture = len([word for word in tagged if word[1] == "MD"])
    numpresent = len([word for word in tagged if word[1] in ["VBP", "VBZ","VBG"]])
    numpast = len([word for word in tagged if word[1] in ["VBD", "VBN"]]) 
    
    return pd.Series([numpast,numpresent,numfuture],index=['NumPast','NumPresent','NumFuture'])

def get_pos_tags(text):
    '''
    Returns part of speech tag counts
    '''
    text = word_tokenize(text)
    tagged = pos_tag(text)

    counts = Counter([word[1] for word in tagged]).items()
    countdict = {}
    for key, value in counts:
        countdict[key] = value  
    
    return countdict

def detect_lang(text):
    '''
    Returns detected language
    '''
    text = doublespace_remove(text)
    try:
        return detect(text)
    except:
        return 'None'
    
def remove_stopwords(row):
    '''
    Multilingual stopwords removal
    '''
    language = row['Language']
    if language in languages:
        text = ' '.join([word for word in row['CleanText'].split(' ') if word not in stopwords.words(language)])
        return text
    else:
        return row['CleanText']
    
def stem_words(row):
    ''' 
    Multilingual word stemmer
    '''
    language = row['Language']    
    if language in languages:
        stemmer = SnowballStemmer(language)
        text = ' '.join([stemmer.stem(word) for word in row['CleanText_NoStop'].split(' ')])
        return text
    else:
        return row['CleanText_NoStop']

In [20]:
def clean_text(df):
    df['Annotation Text'].fillna('',inplace=True)
    df['CleanText'] = df['Annotation Text']
    func_list = [perform_lowercase,replace_numbers,punctuation_remove,underscore_remove, doublespace_remove]
    for func in func_list:
        df['CleanText'] = df['CleanText'].apply(func)

    return df

def featurize(df):
    
    df['AvgWordLength'] = df['CleanText'].apply(get_avg_wordlength)
    df['language_code'] = df['CleanText'].apply(detect_lang)
    df = df.merge(language_codes,how='left',on='language_code')
    df['Language'] = df['Language'].astype('str')
    df['Language'] = df['Language'].apply(lambda x: x.lower().split(';')[0])
    df['Language'] = df['Language'].apply(lambda x: x if x in languages else 'other')
    df['Language'].fillna('None')
    langdummies = pd.get_dummies(df['Language'],prefix='language')
    
    df['CleanText_NoStop'] = df.apply(remove_stopwords,axis=1)
    df['CleanText_NoStop_Stemmed'] = df.apply(stem_words,axis=1)
    df.drop(['language_code','Language'],axis=1,inplace=True)
    
    tenses = df['CleanText'].apply(determine_tense)
    tenses.columns = ['tense_' + col for col in tenses.columns]
    
    postagcounts = []
    for index, row in df.iterrows():
        postagcounts.append(get_pos_tags(row['CleanText']))    
    postagdf = pd.DataFrame(postagcounts).fillna(0)
    postagdf.index = df.index
    postagdf.columns = ['postag_' + col for col in postagdf.columns]
    
    textblobsentdf = df['CleanText'].apply(textblobsent)
    textblobsentdf.columns = ['TextblobPolarity','TextblobSubjectivity']
    df = pd.concat([df,textblobsentdf,tenses,postagdf,langdummies],axis=1)
    
    return df

In [21]:
annotations = clean_text(annotations)
annotations = featurize(annotations)

In [22]:
tfidf_vectorizer = TfidfVectorizer(max_df= .6,
                                 min_df= .002, 
                                 stop_words=None,  
                                 use_idf=True, 
                                 ngram_range=(1,4))
tfidf_matrix = tfidf_vectorizer.fit_transform(annotations['CleanText_NoStop_Stemmed'].values.astype('U'))
terms = tfidf_vectorizer.get_feature_names()
tfidf_matrix = tfidf_matrix.todense()
tfidf = pd.DataFrame(tfidf_matrix)
tfidf.index = annotations.index
tfidf.columns = terms
print tfidf.shape

(16181, 6551)


In [26]:
exclude = ['Source','Category','Topic','Annotation Text','CleanText','CleanText_NoStop','CleanText_NoStop_Stemmed',
           'OCID','PDF Page Number','Article Reference','MD','VBP','VBZ','VBG','VBD','VBN','other',"''",'label','sort_key']
features = [col for col in annotations.columns.tolist() if not col in exclude]

In [27]:
X = pd.concat([tfidf,annotations[features]],axis=1)

In [28]:
X.fillna(0,inplace=True)
X = X.rename(columns = {'fit':'fit_feature'})

In [30]:
classifiers = [
#             LogisticRegression(),
#             DecisionTreeClassifier(),
            RandomForestClassifier()
#             ExtraTreesClassifier()
             ]

In [31]:
parameters = [
#         {'penalty': ['l1','l2']      # LogisticRegression
#               ,'C': [.001,.01,.1,1,10,100]
#               ,'class_weight': [None,'balanced']
#               ,'n_jobs': [-1]}
#         ,{'max_depth': [8,10,12,14]}       #DecisionTree
         {'estimator__n_estimators': [500],          # RandomForest
          'estimator__n_jobs': [-1],
            "estimator__max_depth": [3, 6, 9],
              "estimator__max_features": [.2,.33,.5],
              "estimator__min_samples_split": [2, 4, 10],
              "estimator__min_samples_leaf": [2, 4, 10],
              "estimator__bootstrap": [True, False],
              "estimator__criterion": ["gini", "entropy"],
              "estimator__class_weight": ['balanced','balanced_subsample']}
#         ,{'max_features':[0.2,0.33,0.5]       #ExtraTrees
#               ,'n_estimators': [50]
#               ,'n_jobs': [-1]
#               ,'max_depth': [3,6,9]}
]

In [44]:
def grid_search(X_all, y_all, classifiers, parameters):
    
    # Runs grid search on given list of classifiers and parameters dictionary
    scores = ['recall_macro']
#     X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.33)
    
    results = []
    for i in range(len(classifiers)):
        model = str(classifiers[i]).split('(')[:1][0]
        print '     ******************************************'
        print '     *** ' + model + ' ***'        
        print '     Tuning hyper-parameters for:' 
        for score in scores:
            print '     ' + score.title()
#             clf = GridSearchCV(classifiers[i], parameters[i],cv=n_folds,scoring=score)            
            clf = GridSearchCV(OneVsRestClassifier(classifiers[i]), parameters[i],cv=n_folds,scoring=score)            
            clf.fit(X_all,y_all)
            y_true, y_pred = y_all, clf.predict(X_all)
            
#             acc = accuracy_score(y_true, y_pred)
#             prec = precision_score(y_true, y_pred, average='macro')
#             rec = recall_score(y_true, y_pred, average='macro')
#             roc_auc = roc_auc_score(y_true, y_pred, average='macro')
            print classification_report(y_true, y_pred, target_names=clf.classes_)
            results.append([model, score, clf.best_params_, clf.best_score_])
    return results

In [ ]:
label_results = grid_search(X, y, classifiers, parameters)

     ******************************************
     *** RandomForestClassifier ***
     Tuning hyper-parameters for:
     Recall_Macro


In [ ]:
import pickle
pickle.dump(label_results

In [ ]:
pd.set_option('max_colwidth',200)
resultsdf = pd.DataFrame(label_results)
resultsdf.columns = ['Model','Scoring Method','Best Params','Recall']
resultsdf.to_csv('Multiclass_grid_search_results.csv')
display(resultsdf.sort_values(by='Recall',ascending=False))